In [10]:
library(R.matlab)

In [17]:
df <- readMat('data/spamData.mat')
# Names are: ytrain, ytest, Xtrain, Xtest,
# all of which are matrices

In [128]:
library(pracma)
library(LiblineaR)
library(data.table)

# Separates out the data into K-fold CV chunks.
cv_chunk <- function(x, y, n.folds=10) {
    per.fold <- floor(nrow(x) / n.folds)
    slices <- c(1, seq(n.folds) * per.fold, nrow(x))
    
    # Split into the folds
    x.folds <- lapply(seq(length(slices) - 1), function(s) x[slices[s]:slices[s+1], ])
    y.folds <- lapply(seq(length(slices) - 1), function(s) y[slices[s]:slices[s+1], ])
    
    # Now we need to create the train/test split
    folds <- lapply(seq(length(x.folds)), function(s) {
        train.mask <- seq(length(x.folds)) == s
        test.mask <- !(train.mask)
        
        # Create the training/test sets for each fold
        y.train <- unlist(y.folds[train.mask])
        y.test <- unlist(y.folds[test.mask])
        x.train <- rbindlist(lapply(x.folds[train.mask], as.data.frame))
        x.test <- rbindlist(lapply(x.folds[test.mask], as.data.frame))
        return(list(x.train=x.train, x.test=x.test,
                    y.train=as.matrix(y.train), y.test=as.matrix(y.test)))
    })
    
    return(folds)
}

# Computes the cross-validated MSE for a given model and L2
cv_mse <- function(x, y, cost, n.folds=10) {
    # Shuffle the input row-wise just in case
    rows <- sample(nrow(x))
    x <- x[rows, ]; y <- as.matrix(y[rows, ])
    
    mses <- NULL
    for (fold in cv_chunk(x, y, n.folds=n.folds)) {
        mses <- c(mses, fit_log(fold$x.train, fold$y.train, cost=cost,
                                xnew=fold$x.test, ynew=fold$y.test)$mse)
    }
    
    return(mses)
}

# Fits an L2-regularized logistic regression model and returns
# the fit object and the MSE
fit_log <- function(x, y, cost, xnew=NULL, ynew=NULL) {
    if (is.null(xnew))
        xnew <- x
    if (is.null(ynew))
        ynew <- y
        
    fit <- LiblineaR(x, y, cost=cost)
    return(list(fit=fit, mse=report_mse(fit, xnew, ynew)))
}

# Take a fitted logistic regression model, predict on a dataset
# and report the MSE.
report_mse <- function(fit, x, y) {
    preds <- predict(fit, x, proba=TRUE)$probabilities[, 1]
    mse <- mean((y - preds) ** 2)
    return(mse)
}

# Compute the one-standard-error rule of picking the best
# L2 given a data.frame of L2 and MSEs.
one_se_rule <- function(fits) {
    mse.cutoff <- min(fits$MSE) + (sd(fits$MSE) / sqrt(length(fits$MSE)))
    best.lambda <- fits[which.min(fits$MSE - mse.cutoff), 'L2']
    return(best.lambda)
}

# This function will find the best L2 using CV, and return the fitted model
l2_cv_fit <- function(x, y) {    
    # Compute the MSEs for a bunch of lambdas
    lambdas <- logseq(0.001, 10, 10)
    fits <- rbindlist(lapply(lambdas, function(c) {
        return(data.frame(L2 = c, MSE = mean(cv_mse(x, y, cost=1. / c, n.folds=10))))
    }))
    
    # Fit the best lambda and return the fitted object and MSE
    return(fit_log(x, y, cost=1. / one_se_rule(fits)))
}

## a. Standardize the columns so that they have mean 0 and unit variance

In [131]:
# Compute the means and standard deviations on the training set and use these
# computed values on the test set
mus <- apply(df$Xtrain, 2, mean, na.rm=TRUE)
sds <- apply(df$Xtrain, 2, sd, na.rm=TRUE)

Xtrain <- t(apply(df$Xtrain, 1, function(x) (x - mus) / sds))
Xtest <- t(apply(df$Xtest, 1, function(x) (x - mus) / sds))
    
# Run the analysis
fit <- l2_cv_fit(Xtrain, df$ytrain)$fit
train.mse <- report_mse(fit, Xtrain, df$ytrain)
test.mse <- report_mse(fit, Xtest, df$ytest)
    
print(paste0('Train MSE: ', train.mse, ' test MSE: ', test.mse))

[1] "Train MSE: 0.0554505061520385 test MSE: 0.0612790983849718"


## b. Transform the features using log(x + 0.1)

In [132]:
Xtrain <- log(df$Xtrain + 0.1)
Xtest <- log(df$Xtest + 0.1)
    
# Run the analysis
fit <- l2_cv_fit(Xtrain, df$ytrain)$fit
train.mse <- report_mse(fit, Xtrain, df$ytrain)
test.mse <- report_mse(fit, Xtest, df$ytest)
    
print(paste0('Train MSE: ', train.mse, ' test MSE: ', test.mse))

[1] "Train MSE: 0.0411775519387855 test MSE: 0.045006475915237"


## c. Binarize the features using I(x > 0)

In [134]:
Xtrain <- apply(df$Xtrain, 2, function(x) as.numeric(x > 0))
Xtest <- apply(df$Xtest, 2, function(x) as.numeric(x > 0))

# Run the analysis
fit <- l2_cv_fit(Xtrain, df$ytrain)$fit
train.mse <- report_mse(fit, Xtrain, df$ytrain)
test.mse <- report_mse(fit, Xtest, df$ytest)
    
print(paste0('Train MSE: ', train.mse, ' test MSE: ', test.mse))

[1] "Train MSE: 0.0557650962537647 test MSE: 0.0607532968837769"
